In [8]:
import numpy as np
import pandas as pd
import sqlalchemy as sqlal
import requests
def configure_api(date, secretkey, zipcode, line_up_id, datetm):
    final_link_theatre = "http://data.tmsapi.com/v1.1/movies/showings?startDate=" + date + "&zip=" + zipcode + "&api_key=" + secretkey
    final_link_airing = "http://data.tmsapi.com/v1.1/movies/airings?lineupId=" + line_up_id +  "&startDateTime=" + date +  "&api_key=" + secretkey
    resp_theatre = requests.get(final_link_theatre)
    resp_airing = requests.get(final_link_airing)
    return resp_theatre, resp_airing

def fetch_data_from_api_and_load_into_sql(req_api, table_name, engine, flag):
    if req_api.text!='':
        df = pd.DataFrame.from_dict(req_api.json())
        if flag=="channel":
            df = pd.concat([df.drop('program', axis=1), pd.DataFrame(df['program'].tolist())], axis=1)
        df = df[["title", "genres", "shortDescription", "releaseYear"]].copy()
        df = df.explode("genres")
        df["theatreOrChannel"] = flag
    else:
        print("Invalid data")
    df.to_sql(table_name, con=engine)

def analysis_on_data(sql_engine, table1, table2):
    df_theatre = pd.read_sql_table(table1, sql_engine)
    df_channel = pd.read_sql_table(table2, sql_engine)
    print("Top 5 genre counts on theatre:\n")
    print(df_theatre.genres.value_counts().nlargest(5))
    print("Top 5 genre counts on channel:\n")
    print(df_channel.genres.value_counts().nlargest(5))
    df_combined = df_theatre.append(df_channel, ignore_index=True)
    print(df_combined.genres.value_counts().nlargest(5))
    df_max_genres = df_combined[df_combined["genres"].isin(df_combined.genres.value_counts().nlargest(5).index.to_list())]
    return df_max_genres
                                        
if __name__ == '__main__':
    engine = sqlal.create_engine('sqlite:///:memory:')
    resp_theatre, resp_airing = configure_api("2020-11-18", "eyejfpj8v2ssrvvdzrccwgpe", "78701", "USA-TX42500-X", "2020-11-18T09:30Z")
    fetch_data_from_api_and_load_into_sql(resp_theatre, "theatre",engine, "theatre")
    fetch_data_from_api_and_load_into_sql(resp_airing, "channel",engine, "channel")
    df_max_genres_count = analysis_on_data(engine, "theatre", "channel")
    print("Top 5 genre in combined theatre and channel are:\n")
    print(df_max_genres_count.genres.value_counts()) 
    
    
    

Top 5 genre counts on theatre:

Comedy             5
Thriller           5
Action             3
Science fiction    3
Adventure          2
Name: genres, dtype: int64
Top 5 genre counts on channel:

Action       88
Comedy       80
Drama        72
Thriller     70
Adventure    49
Name: genres, dtype: int64
Action       91
Comedy       85
Thriller     75
Drama        72
Adventure    51
Name: genres, dtype: int64
Top 5 genre in combined theatre and channel are:

Action       91
Comedy       85
Thriller     75
Drama        72
Adventure    51
Name: genres, dtype: int64


In [13]:
import json
meta = sqlal.MetaData()
meta.reflect(bind=engine)  # http://docs.sqlalchemy.org/en/rel_0_9/core/reflection.html
result = {}
for table in meta.sorted_tables:
    result[table.name] = [dict(row) for row in engine.execute(table.select())]
with open('data.json', 'w') as outfile:
    json.dump(result, outfile)